In [2]:
# %%capture
# Paste this cell in your notebook. It contains plotting utilities.
import re
import os
from pathlib import Path
from typing import Optional, Sequence, Union, Tuple, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams.update({"figure.dpi": 150, "axes.grid": True})

# -------------------------
# Helper parsing functions
# -------------------------
def parse_run_name(name: str) -> Tuple[str, Union[int,str]]:
    """
    Parse a run name like:
      'test_sampled_uniform_1_J_4' -> shard_key='sampled_uniform_1', J=4
      'test_iid_J_4'              -> shard_key='iid', J=4
      'test_non_iid_10_J_8'      -> shard_key='non_iid_10', J=8
    Returns (shard_key, J)
    """
    if not isinstance(name, str):
        return ("unknown", -1)
    m = re.match(r'^(?:test_)?(.+)_J_(\d+)$', name)
    if m:
        shard_key = m.group(1)
        J = int(m.group(2))
        return shard_key, J
    # fallback: try to find _J_ anywhere
    m2 = re.search(r'_J_(\d+)', name)
    if m2:
        J = int(m2.group(1))
        # try to extract shard_key before _J_
        before = name.split('_J_')[0]
        if before.startswith("test_"):
            shard_key = before[len("test_"):]
        else:
            shard_key = before
        return shard_key, J
    return (name, -1)

def shardkey_to_Nc_label(shard_key: str) -> str:
    """
    Convert shard_key to a compact Nc label:
      'sampled_uniform_1' -> 'Nc=1'
      'sampled_uniform_50' -> 'Nc=50'
      'iid' -> 'iid'
      fallback returns shard_key
    """
    if shard_key is None:
        return 'unknown'
    if shard_key.lower().startswith('iid'):
        return 'iid'
    # try to find trailing number
    m = re.search(r'_(\d+)$', shard_key)
    if m:
        return f"Nc={m.group(1)}"
    # try internal number
    m2 = re.search(r'(\d+)', shard_key)
    if m2:
        return f"Nc={m2.group(1)}"
    return shard_key

# -------------------------
# Loading & preprocessing
# -------------------------
def load_experiment_csv(csv_path: Union[str, Path]) -> pd.DataFrame:
    """
    Load CSV and add parsed columns:
      - 'shard_key'  (e.g., sampled_uniform_1 or iid)
      - 'Nc_label'   (e.g., Nc=1 or iid)
      - 'J'          (int)
    Returns dataframe with these columns appended.
    """
    df = pd.read_csv(csv_path)
    # normalize column names
    df.columns = [c.strip() for c in df.columns]
    if 'name' not in df.columns:
        raise ValueError("CSV must contain column 'name' (run identifier).")
    # parse
    parsed = df['name'].apply(parse_run_name)
    df['shard_key'] = parsed.apply(lambda x: x[0])
    df['J'] = parsed.apply(lambda x: x[1])
    df['Nc_label'] = df['shard_key'].apply(shardkey_to_Nc_label)
    # ensure numeric columns are floats
    for col in ['round','val_loss','val_acc','test_loss','test_acc','train_loss','train_acc','local_train_mean','local_train_std']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# -------------------------
# Aggregation helpers
# -------------------------
def final_metrics_per_run(df: pd.DataFrame) -> pd.DataFrame:
    """
    For every unique 'name' (run) return the last-round metrics.
    Returns DataFrame indexed by 'name' with columns including 'test_acc', 'J', 'Nc_label', 'shard_key', 'round'
    """
    out_rows = []
    for name, group in df.groupby('name'):
        # pick the row with max round (if round exists), else last row
        if 'round' in group.columns and not group['round'].isna().all():
            row = group.loc[group['round'].idxmax()]
        else:
            row = group.iloc[-1]
        d = {
            'name': name,
            'shard_key': row.get('shard_key'),
            'Nc_label': row.get('Nc_label'),
            'J': int(row.get('J', -1)),
            'final_round': row.get('round'),
            'test_acc': row.get('test_acc'),
            'val_acc': row.get('val_acc'),
            'test_loss': row.get('test_loss'),
            'val_loss': row.get('val_loss')
        }
        out_rows.append(d)
    out_df = pd.DataFrame(out_rows)
    return out_df

# -------------------------
# Plotting / Reporting
# -------------------------
def ensure_outdir(out_dir: Union[str, Path]):
    Path(out_dir).mkdir(parents=True, exist_ok=True)

def plot_accuracy_vs_rounds(df: pd.DataFrame, out_dir: str,
                            nc_order: Optional[Sequence[str]] = None,
                            J_list: Optional[Sequence[int]] = (4,8,16),
                            smoothing: int = 1):
    """
    Plot test accuracy vs rounds grouped by Nc label.
    - df: loaded DataFrame from load_experiment_csv
    - out_dir: directory to save PNGs
    - nc_order: list of Nc_label strings to iterate in order, e.g. ['Nc=1','Nc=5','Nc=10','Nc=50','iid']
    - smoothing: optional moving average kernel (integer)
    """
    ensure_outdir(out_dir)
    if nc_order is None:
        nc_order = sorted(df['Nc_label'].unique(), key=lambda x: (x!='iid', x))
    summary = {}
    for nc in nc_order:
        plt.figure(figsize=(6,4))
        ax = plt.gca()
        subset = df[df['Nc_label']==nc]
        if subset.empty:
            print(f"Warning: no data for {nc}, skipping.")
            continue
        for J in J_list:
            sel = subset[subset['J']==J]
            if sel.empty:
                continue
            # group across runs (names) by round -> mean/std
            grouped = sel.groupby('round')['test_acc'].agg(['mean','std','count']).reset_index()
            if grouped.empty:
                continue
            x = grouped['round'].values
            y = grouped['mean'].values
            if smoothing > 1:
                # simple moving average
                kernel = np.ones(smoothing) / smoothing
                y = np.convolve(y, kernel, mode='same')
            ax.plot(x, y, label=f"J={J}")
            # shaded std if available
            if 'std' in grouped.columns and not grouped['std'].isna().all():
                std = grouped['std'].values
                ax.fill_between(x, y-std, y+std, alpha=0.2)
        ax.set_title(f"Test accuracy vs rounds — {nc}")
        ax.set_xlabel("Round")
        ax.set_ylabel("Test accuracy")
        ax.legend()
        ax.set_ylim(0.0, 1.0)
        plt.tight_layout()
        out_file = Path(out_dir)/f"acc_vs_rounds_{nc.replace('=','_')}.png"
        plt.savefig(out_file)
        plt.close()
        summary[nc] = str(out_file)
    print("Saved accuracy-vs-rounds plots:", summary)
    return summary

def plot_final_acc_vs_J(df: pd.DataFrame, out_dir: str,
                        nc_order: Optional[Sequence[str]] = None,
                        J_list: Optional[Sequence[int]] = (4,8,16)):
    """
    Bar plot showing final test accuracy (mean ± std) vs J for each Nc.
    """
    ensure_outdir(out_dir)
    final = final_metrics_per_run(df)
    # aggregate per (Nc_label, J)
    agg = final.groupby(['Nc_label','J'])['test_acc'].agg(['mean','std','count']).reset_index()
    if nc_order is None:
        nc_order = sorted(agg['Nc_label'].unique(), key=lambda x: (x!='iid', x))
    fig, ax = plt.subplots(figsize=(7,4))
    width = 0.15
    x_positions = np.arange(len(nc_order))
    offsets = np.linspace(-width, width, num=len(J_list))
    for i, J in enumerate(J_list):
        vals = []
        errs = []
        for nc in nc_order:
            row = agg[(agg['Nc_label']==nc) & (agg['J']==J)]
            if not row.empty:
                vals.append(float(row['mean']))
                errs.append(float(row['std']) if not np.isnan(row['std'].iloc[0]) else 0.0)
            else:
                vals.append(0.0)
                errs.append(0.0)
        ax.bar(x_positions + offsets[i], vals, width=width, yerr=errs, label=f"J={J}", capsize=3)
    ax.set_xticks(x_positions)
    ax.set_xticklabels(nc_order)
    ax.set_ylim(0,1.0)
    ax.set_ylabel("Final test accuracy (mean ± std)")
    ax.set_title("Final test accuracy vs J for each Nc")
    ax.legend()
    plt.tight_layout()
    out_file = Path(out_dir)/"final_acc_vs_J.png"
    plt.savefig(out_file)
    plt.close()
    print("Saved final-accuracy-vs-J plot:", out_file)
    return str(out_file)

def export_final_metrics_csv(df: pd.DataFrame, out_path: Union[str, Path]):
    """
    Export a CSV with final metrics per run and aggregated stats per (Nc,J).
    """
    final = final_metrics_per_run(df)
    # save per-run final
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    final.to_csv(out_path, index=False)
    # also save aggregated summary
    agg = final.groupby(['Nc_label','J'])['test_acc'].agg(['mean','std','count']).reset_index()
    agg.to_csv(out_path.parent / (out_path.stem + "_aggregated.csv"), index=False)
    print("Exported final per-run and aggregated CSVs to:", out_path)
    return final, agg

# -------------------------
# Optional: heatmaps / per-client plots (if you have per-client arrays)
# -------------------------
def plot_shard_heatmap(client_class_counts: np.ndarray,
                       out_file: Union[str, Path],
                       client_labels: Optional[Sequence[str]] = None,
                       class_labels: Optional[Sequence[str]] = None,
                       cmap: str = "viridis"):
    """
    client_class_counts: 2D array shape (n_clients, n_classes) with counts.
    """
    ensure_outdir(Path(out_file).parent)
    counts = np.array(client_class_counts)
    plt.figure(figsize=(8,6))
    plt.imshow(counts, aspect='auto', interpolation='nearest', cmap=cmap)
    plt.colorbar(label="sample count")
    if client_labels is not None:
        # too many ticks -> subsample
        plt.yticks(np.linspace(0, counts.shape[0]-1, min(10, counts.shape[0])).astype(int),
                   [client_labels[i] for i in np.linspace(0, counts.shape[0]-1, min(10, counts.shape[0])).astype(int)])
    if class_labels is not None:
        plt.xticks(np.linspace(0, counts.shape[1]-1, min(10, counts.shape[1])).astype(int),
                   [class_labels[i] for i in np.linspace(0, counts.shape[1]-1, min(10, counts.shape[1])).astype(int)],
                   rotation=45, ha='right')
    plt.xlabel("Class")
    plt.ylabel("Client")
    plt.title("Client × Class sample counts")
    plt.tight_layout()
    plt.savefig(out_file)
    plt.close()
    print("Saved shard heatmap to", out_file)
    return str(out_file)

def plot_per_client_accuracy_hist(per_client_acc: np.ndarray, out_file: Union[str, Path],
                                  bins: int = 20):
    ensure_outdir(Path(out_file).parent)
    arr = np.array(per_client_acc)
    plt.figure(figsize=(6,4))
    plt.hist(arr, bins=bins, edgecolor='k')
    plt.xlabel("Per-client test accuracy")
    plt.ylabel("Number of clients")
    plt.title("Per-client accuracy distribution (final round)")
    plt.tight_layout()
    plt.savefig(out_file)
    plt.close()
    print("Saved per-client histogram to", out_file)
    return str(out_file)

# -------------------------
# Orchestrator: produce all standard plots
# -------------------------
def generate_all_plots(csv_path: str, out_dir: str,
                       nc_order: Optional[Sequence[str]] = None,
                       J_list: Optional[Sequence[int]] = (4,8,16),
                       smoothing: int = 1):
    """
    Top-level convenience function to load the CSV and produce:
      - accuracy vs rounds per Nc (one file per Nc)
      - final accuracy vs J (single panel)
      - export final metrics CSVs
    Returns a dict of saved file paths.
    """
    df = load_experiment_csv(csv_path)
    ensure_outdir(out_dir)
    # default ordering for Nc
    if nc_order is None:
        # prefer Nc=1,5,10,50, iid ordering if present
        possible = ['Nc=1','Nc=5','Nc=10','Nc=50','iid']
        present = [x for x in possible if x in df['Nc_label'].unique()]
        other = []
        nc_order = present + other
    summary_paths = {}
    summary_paths.update(plot_accuracy_vs_rounds(df, out_dir, nc_order=nc_order, J_list=J_list, smoothing=smoothing))
    summary_paths['final_acc_vs_J'] = plot_final_acc_vs_J(df, out_dir, nc_order=nc_order, J_list=J_list)
    final_df, agg_df = export_final_metrics_csv(df, Path(out_dir)/"final_metrics_per_run.csv")
    summary_paths['final_metrics_csv'] = str(Path(out_dir)/"final_metrics_per_run.csv")
    summary_paths['aggregated_csv'] = str(Path(out_dir)/"final_metrics_per_run_aggregated.csv")
    print("All done. Summary:", summary_paths)
    return summary_paths

# -------------------------
# Example usage (uncomment & edit paths)
# -------------------------
CSV_PATH = "le_nouveaux_log.csv"   # <--- set your CSV path
OUT_DIR = "figs/auto_plots"
generate_all_plots(CSV_PATH, OUT_DIR, smoothing=3)
#
# If you have per-client arrays (numpy) available, call:
#plot_shard_heatmap(my_client_class_counts, "figs/auto_plots/shard_heatmap.png")
#plot_per_client_accuracy_hist(my_per_client_acc_array, "figs/auto_plots/per_client_hist.png")
#
# To inspect a quick summary table:
# df = load_experiment_csv(CSV_PATH)
# final = final_metrics_per_run(df)
# print(final.groupby(['Nc_label','J'])['test_acc'].agg(['mean','std','count']).reset_index())


Saved accuracy-vs-rounds plots: {'Nc=1': 'figs\\auto_plots\\acc_vs_rounds_Nc_1.png', 'Nc=5': 'figs\\auto_plots\\acc_vs_rounds_Nc_5.png', 'Nc=10': 'figs\\auto_plots\\acc_vs_rounds_Nc_10.png', 'Nc=50': 'figs\\auto_plots\\acc_vs_rounds_Nc_50.png', 'iid': 'figs\\auto_plots\\acc_vs_rounds_iid.png'}
Saved final-accuracy-vs-J plot: figs\auto_plots\final_acc_vs_J.png
Exported final per-run and aggregated CSVs to: figs\auto_plots\final_metrics_per_run.csv
All done. Summary: {'Nc=1': 'figs\\auto_plots\\acc_vs_rounds_Nc_1.png', 'Nc=5': 'figs\\auto_plots\\acc_vs_rounds_Nc_5.png', 'Nc=10': 'figs\\auto_plots\\acc_vs_rounds_Nc_10.png', 'Nc=50': 'figs\\auto_plots\\acc_vs_rounds_Nc_50.png', 'iid': 'figs\\auto_plots\\acc_vs_rounds_iid.png', 'final_acc_vs_J': 'figs\\auto_plots\\final_acc_vs_J.png', 'final_metrics_csv': 'figs\\auto_plots\\final_metrics_per_run.csv', 'aggregated_csv': 'figs\\auto_plots\\final_metrics_per_run_aggregated.csv'}


C:\Users\matti\AppData\Local\Temp\ipykernel_30524\4182015453.py:206: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  vals.append(float(row['mean']))
C:\Users\matti\AppData\Local\Temp\ipykernel_30524\4182015453.py:207: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  errs.append(float(row['std']) if not np.isnan(row['std'].iloc[0]) else 0.0)


{'Nc=1': 'figs\\auto_plots\\acc_vs_rounds_Nc_1.png',
 'Nc=5': 'figs\\auto_plots\\acc_vs_rounds_Nc_5.png',
 'Nc=10': 'figs\\auto_plots\\acc_vs_rounds_Nc_10.png',
 'Nc=50': 'figs\\auto_plots\\acc_vs_rounds_Nc_50.png',
 'iid': 'figs\\auto_plots\\acc_vs_rounds_iid.png',
 'final_acc_vs_J': 'figs\\auto_plots\\final_acc_vs_J.png',
 'final_metrics_csv': 'figs\\auto_plots\\final_metrics_per_run.csv',
 'aggregated_csv': 'figs\\auto_plots\\final_metrics_per_run_aggregated.csv'}

In [3]:
# -------------------------
# Loss plotting utilities (drop-in)
# -------------------------
def plot_loss_vs_rounds(df: pd.DataFrame, out_dir: str,
                        nc_order: Optional[Sequence[str]] = None,
                        J_list: Optional[Sequence[int]] = (4,8,16),
                        smoothing: int = 1):
    """
    Plot test loss vs rounds grouped by Nc label.
    - df: DataFrame from load_experiment_csv
    - out_dir: directory to save PNGs
    - nc_order: list of Nc_label strings to iterate in order
    - smoothing: optional moving average kernel (integer)
    """
    ensure_outdir(out_dir)
    if nc_order is None:
        nc_order = sorted(df['Nc_label'].unique(), key=lambda x: (x!='iid', x))
    summary = {}
    for nc in nc_order:
        plt.figure(figsize=(6,4))
        ax = plt.gca()
        subset = df[df['Nc_label']==nc]
        if subset.empty:
            print(f"Warning: no data for {nc}, skipping.")
            continue
        for J in J_list:
            sel = subset[subset['J']==J]
            if sel.empty:
                continue
            grouped = sel.groupby('round')['test_loss'].agg(['mean','std','count']).reset_index()
            if grouped.empty:
                continue
            x = grouped['round'].values
            y = grouped['mean'].values
            if smoothing > 1:
                kernel = np.ones(smoothing) / smoothing
                y = np.convolve(y, kernel, mode='same')
            ax.plot(x, y, label=f"J={J}")
            if 'std' in grouped.columns and not grouped['std'].isna().all():
                std = grouped['std'].values
                ax.fill_between(x, y-std, y+std, alpha=0.2)
        ax.set_title(f"Test loss vs rounds — {nc}")
        ax.set_xlabel("Round")
        ax.set_ylabel("Test loss")
        ax.legend()
        plt.tight_layout()
        out_file = Path(out_dir)/f"loss_vs_rounds_{nc.replace('=','_')}.png"
        plt.savefig(out_file)
        plt.close()
        summary[nc] = str(out_file)
    print("Saved loss-vs-rounds plots:", summary)
    return summary

def plot_final_loss_vs_J(df: pd.DataFrame, out_dir: str,
                         nc_order: Optional[Sequence[str]] = None,
                         J_list: Optional[Sequence[int]] = (4,8,16)):
    """
    Bar plot showing final test loss (mean ± std) vs J for each Nc.
    """
    ensure_outdir(out_dir)
    final = final_metrics_per_run(df)
    # aggregate per (Nc_label, J)
    # note: final_metrics_per_run returns test_loss in addition to test_acc
    agg = final.groupby(['Nc_label','J'])['test_loss'].agg(['mean','std','count']).reset_index()
    if nc_order is None:
        nc_order = sorted(agg['Nc_label'].unique(), key=lambda x: (x!='iid', x))
    fig, ax = plt.subplots(figsize=(7,4))
    width = 0.15
    x_positions = np.arange(len(nc_order))
    offsets = np.linspace(-width, width, num=len(J_list))
    for i, J in enumerate(J_list):
        vals = []
        errs = []
        for nc in nc_order:
            row = agg[(agg['Nc_label']==nc) & (agg['J']==J)]
            if not row.empty:
                vals.append(float(row['mean']))
                errs.append(float(row['std']) if not np.isnan(row['std'].iloc[0]) else 0.0)
            else:
                vals.append(0.0)
                errs.append(0.0)
        ax.bar(x_positions + offsets[i], vals, width=width, yerr=errs, label=f"J={J}", capsize=3)
    ax.set_xticks(x_positions)
    ax.set_xticklabels(nc_order)
    ax.set_ylabel("Final test loss (mean ± std)")
    ax.set_title("Final test loss vs J for each Nc")
    ax.legend()
    plt.tight_layout()
    out_file = Path(out_dir)/"final_loss_vs_J.png"
    plt.savefig(out_file)
    plt.close()
    print("Saved final-loss-vs-J plot:", out_file)
    return str(out_file)

def export_final_metrics_csv_with_loss(df: pd.DataFrame, out_path: Union[str, Path]):
    """
    Export a CSV with final metrics per run including loss and aggregated stats per (Nc,J).
    """
    final = final_metrics_per_run(df)
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    final.to_csv(out_path, index=False)
    # aggregated summary for both acc and loss
    agg_acc = final.groupby(['Nc_label','J'])['test_acc'].agg(['mean','std','count']).reset_index()
    agg_loss = final.groupby(['Nc_label','J'])['test_loss'].agg(['mean','std','count']).reset_index()
    agg_acc.to_csv(out_path.parent / (out_path.stem + "_aggregated_acc.csv"), index=False)
    agg_loss.to_csv(out_path.parent / (out_path.stem + "_aggregated_loss.csv"), index=False)
    print("Exported final per-run and aggregated CSVs to:", out_path)
    return final, agg_acc, agg_loss

def generate_all_plots_with_loss(csv_path: str, out_dir: str,
                                 nc_order: Optional[Sequence[str]] = None,
                                 J_list: Optional[Sequence[int]] = (4,8,16),
                                 smoothing: int = 1):
    """
    Convenience wrapper to create accuracy + loss plots and exports.
    """
    df = load_experiment_csv(csv_path)
    ensure_outdir(out_dir)
    if nc_order is None:
        possible = ['Nc=1','Nc=5','Nc=10','Nc=50','iid']
        present = [x for x in possible if x in df['Nc_label'].unique()]
        other = []
        nc_order = present + other
    summary_paths = {}
    # accuracy
    summary_paths.update(plot_accuracy_vs_rounds(df, out_dir, nc_order=nc_order, J_list=J_list, smoothing=smoothing))
    summary_paths['final_acc_vs_J'] = plot_final_acc_vs_J(df, out_dir, nc_order=nc_order, J_list=J_list)
    # loss
    summary_paths.update(plot_loss_vs_rounds(df, out_dir, nc_order=nc_order, J_list=J_list, smoothing=smoothing))
    summary_paths['final_loss_vs_J'] = plot_final_loss_vs_J(df, out_dir, nc_order=nc_order, J_list=J_list)
    # exports
    final_df, agg_acc, agg_loss = export_final_metrics_csv_with_loss(df, Path(out_dir)/"final_metrics_per_run_with_loss.csv")
    summary_paths['final_metrics_csv'] = str(Path(out_dir)/"final_metrics_per_run_with_loss.csv")
    summary_paths['aggregated_acc_csv'] = str(Path(out_dir)/"final_metrics_per_run_with_loss_aggregated_acc.csv")
    summary_paths['aggregated_loss_csv'] = str(Path(out_dir)/"final_metrics_per_run_with_loss_aggregated_loss.csv")
    print("All done. Summary:", summary_paths)
    return summary_paths

# -------------------------
# Example usage (uncomment & edit paths)
# -------------------------
CSV_PATH = "le_nouveaux_log.csv"   # <--- set your CSV path
OUT_DIR = "figs/auto_plots"
generate_all_plots_with_loss(CSV_PATH, OUT_DIR, smoothing=3)


Saved accuracy-vs-rounds plots: {'Nc=1': 'figs\\auto_plots\\acc_vs_rounds_Nc_1.png', 'Nc=5': 'figs\\auto_plots\\acc_vs_rounds_Nc_5.png', 'Nc=10': 'figs\\auto_plots\\acc_vs_rounds_Nc_10.png', 'Nc=50': 'figs\\auto_plots\\acc_vs_rounds_Nc_50.png', 'iid': 'figs\\auto_plots\\acc_vs_rounds_iid.png'}
Saved final-accuracy-vs-J plot: figs\auto_plots\final_acc_vs_J.png


C:\Users\matti\AppData\Local\Temp\ipykernel_30524\4182015453.py:206: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  vals.append(float(row['mean']))
C:\Users\matti\AppData\Local\Temp\ipykernel_30524\4182015453.py:207: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  errs.append(float(row['std']) if not np.isnan(row['std'].iloc[0]) else 0.0)


Saved loss-vs-rounds plots: {'Nc=1': 'figs\\auto_plots\\loss_vs_rounds_Nc_1.png', 'Nc=5': 'figs\\auto_plots\\loss_vs_rounds_Nc_5.png', 'Nc=10': 'figs\\auto_plots\\loss_vs_rounds_Nc_10.png', 'Nc=50': 'figs\\auto_plots\\loss_vs_rounds_Nc_50.png', 'iid': 'figs\\auto_plots\\loss_vs_rounds_iid.png'}
Saved final-loss-vs-J plot: figs\auto_plots\final_loss_vs_J.png
Exported final per-run and aggregated CSVs to: figs\auto_plots\final_metrics_per_run_with_loss.csv
All done. Summary: {'Nc=1': 'figs\\auto_plots\\loss_vs_rounds_Nc_1.png', 'Nc=5': 'figs\\auto_plots\\loss_vs_rounds_Nc_5.png', 'Nc=10': 'figs\\auto_plots\\loss_vs_rounds_Nc_10.png', 'Nc=50': 'figs\\auto_plots\\loss_vs_rounds_Nc_50.png', 'iid': 'figs\\auto_plots\\loss_vs_rounds_iid.png', 'final_acc_vs_J': 'figs\\auto_plots\\final_acc_vs_J.png', 'final_loss_vs_J': 'figs\\auto_plots\\final_loss_vs_J.png', 'final_metrics_csv': 'figs\\auto_plots\\final_metrics_per_run_with_loss.csv', 'aggregated_acc_csv': 'figs\\auto_plots\\final_metrics_per

C:\Users\matti\AppData\Local\Temp\ipykernel_30524\3709557724.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  vals.append(float(row['mean']))
C:\Users\matti\AppData\Local\Temp\ipykernel_30524\3709557724.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  errs.append(float(row['std']) if not np.isnan(row['std'].iloc[0]) else 0.0)


{'Nc=1': 'figs\\auto_plots\\loss_vs_rounds_Nc_1.png',
 'Nc=5': 'figs\\auto_plots\\loss_vs_rounds_Nc_5.png',
 'Nc=10': 'figs\\auto_plots\\loss_vs_rounds_Nc_10.png',
 'Nc=50': 'figs\\auto_plots\\loss_vs_rounds_Nc_50.png',
 'iid': 'figs\\auto_plots\\loss_vs_rounds_iid.png',
 'final_acc_vs_J': 'figs\\auto_plots\\final_acc_vs_J.png',
 'final_loss_vs_J': 'figs\\auto_plots\\final_loss_vs_J.png',
 'final_metrics_csv': 'figs\\auto_plots\\final_metrics_per_run_with_loss.csv',
 'aggregated_acc_csv': 'figs\\auto_plots\\final_metrics_per_run_with_loss_aggregated_acc.csv',
 'aggregated_loss_csv': 'figs\\auto_plots\\final_metrics_per_run_with_loss_aggregated_loss.csv'}